# Cohere Tools

The following notebook goes over how to use the Cohere tools agent:

Prerequisites for this notebook:

# Environment

In [ ]:
! pip install --quiet langchain langchain_experimental langchain_cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 4.1 MB/s eta 0:00:00


In [ ]:
! pip install --force-reinstall --quiet 'git+https://github.com/BeatrixCohere/langchain.git@beatrix/MultiHopAgent#subdirectory=libs/partners/cohere&egg=langchain-cohere'

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 13.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the so

In [8]:
### LLMs
import os
COHERE_API_KEY = "a344FMSW0GsEpjLdqov1QK1jIJ4g2yg1BQ7RAzl9" # <your-api-key>
# TAVILY_API_KEY = "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" # <your-api-key>

# Define tools

### Web search tool

In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults

os.environ['TAVILY_API_KEY'] = "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"
web_search_tool = TavilySearchResults(max_results=3)


### Python interpreter tool

In [10]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()

repl_tool = Tool(
    name="python_repl",
    description="Executes python code and returns the result. The code runs in a static sandbox without interactive mode, so print output or save output to a file.",
    func=python_repl.run,
)


### A tool based on a function

In [11]:
# demo how to easily transform a python function into a tool ?


# Create ReAct Agent

In [12]:
from langchain.agents import AgentExecutor
from langchain_cohere.chat_models import ChatCohere
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate

In [13]:
# Create the prompt
prompt = ChatPromptTemplate.from_template(
    "{input}"
)

# Create the Cohere chat model
chat = ChatCohere(cohere_api_key=COHERE_API_KEY, model="command-r")

# Create the ReAct agent
agent = create_cohere_react_agent(
    llm=chat,
    tools=[web_search_tool, repl_tool],
    prompt=prompt,
)

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=[web_search_tool, repl_tool], verbose=True)
# agent_executor = AgentExecutor(agent=agent, tools=[web_search_tool], verbose=True)
agent_executor.invoke({"input": "Create a plot of the number of full time employees at the three most valuable"})



> Entering new AgentExecutor chain...
Calling: {'tool_name': 'tavily_search_results_json', 'parameters': {'query': 'three most valuable companies number of employees'}} with plan I will search for the three most valuable companies and their number of full-time employees, then write and execute Python code to plot the data.[{'url': 'https://www.visualcapitalist.com/profit-per-employee-top-u-s-companies-ranking/', 'content': "That said, the top 10 within this cohort boasted an average profit per employee of $1.13 million, while the bottom 10 companies sit at an average of $348,300. Of course, companies not making the ranking would have even more modest numbers. For example, it's estimated that Starbucks makes around $80,000 per employee, and Coca-Cola makes around ..."}, {'url': 'https://www.statista.com/statistics/264671/top-50-companies-based-on-number-of-employees/', 'content': 'Largest companies in the world based on revenues, by number of employees 2022\nAmazon\nLeading 500 Fortun